> ## Import Package

In [1]:
import os
import torch
import numpy as np
import pandas as pd
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

> ## Define Constants
> If DEBUG_TRAIN is true, shows the shape of the tensor during the learning process.<br>
> If DEBUG_DATA is true, shows the data during the learning process.<br>
> MODEL_PATH specifies the path, which models completed learning will be stored.

In [2]:
DEBUG_TRAIN = False
DEBUG_DATA = False

MODEL_PATH = './saved_model/BCEWithLogitsLoss-Complete.torchModel'
CSV_FILE = "./image-data/labels-map.csv"
ROOT_DIR = "./image-data/hangul-images"

batch_size=64
learning_rate=0.002
num_epoch=100

> ## Define Functions
> **encode_from_utf8**: returns a list of bit that converted from character represented by UTF-8<br>
> **decode_from_bin**: returns a character represented by UTF-8 that converted from list of bit<br>

In [3]:
def encode_from_utf8(x):
    return list(map(float,bin(int(x.encode().hex(),16))[2:]))

In [4]:
def decode_from_bin(x):
    res = ""
    for i in x:
        res+=str(round(i))
        pass
    return bytearray.fromhex(hex(int(res, 2))[2:]).decode()

In [5]:
def decode_from_bin2(x):
    res = ""
    for i in x:
        res+=str(int(i>0))
        pass
    return bytearray.fromhex(hex(int(res, 2))[2:]).decode()

> ## Define Dataset Class

In [6]:
class KoreanHandwritingDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.dataset = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = self.dataset.iloc[idx, 0]
        image = io.imread(img_name)
        label = self.dataset.iloc[idx, 1]
        label = np.array([label])
        sample = {'image': image, 'label': label}

        if self.transform:
            sample = self.transform(sample)

        return sample
    pass

> ## Define Transform Class

In [7]:
class ToTensor(object):
    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        image = image/255.0
        return {'image': torch.from_numpy(image.reshape(1,64,64)),
                'label': torch.tensor(encode_from_utf8(label[0]))}

In [8]:
korean_dataset = KoreanHandwritingDataset(CSV_FILE, ROOT_DIR, transform=transforms.Compose([ToTensor()]))
dataloader = DataLoader(korean_dataset, batch_size = batch_size, shuffle = True, num_workers = 0)

> ## Define Model

In [9]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.avgpool1 = nn.AvgPool2d(2,2) # kernel size 2x2 (32 = 64/2)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(2,2) # kernel size 2x2 (32 = 64/2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(2,2) # kernel size 2x2 (16 = 32/2)
        self.linear1 = nn.Linear(1152,378)
        self.relu3 = nn.ReLU()
        self.linear2 = nn.Linear(378,128)
        self.relu4 = nn.ReLU()
        self.linear3 = nn.Linear(128,24)
        #self.sigmoid = nn.Sigmoid()
        pass
    def forward(self,x):
        out = self.avgpool1(x)
        if DEBUG_TRAIN : print('avgpool1: ', out.shape)
        out = self.conv1(out)
        if DEBUG_TRAIN : print('conv1: ', out.shape)
        out = self.relu1(out)
        if DEBUG_TRAIN : print('relu1: ', out.shape)
        out = self.maxpool1(out)
        if DEBUG_TRAIN : print('maxpool1: ', out.shape)
        out = self.conv2(out)
        if DEBUG_TRAIN : print('conv2: ', out.shape)
        out = self.relu2(out)
        if DEBUG_TRAIN : print('relu2: ', out.shape)
        out = self.maxpool2(out)
        if DEBUG_TRAIN : print('maxpool2: ', out.shape)
        out = out.view(out.size(0),-1)
        if DEBUG_TRAIN : print('view: ', out.shape)
        out = self.linear1(out)
        if DEBUG_TRAIN : print('linear1: ', out.shape)
        out = self.relu3(out)
        if DEBUG_TRAIN : print('relu3: ', out.shape)
        out = self.linear2(out)
        if DEBUG_TRAIN : print('linear2: ', out.shape)
        out = self.relu4(out)
        if DEBUG_TRAIN : print('relu4: ', out.shape)
        out = self.linear3(out)
        if DEBUG_TRAIN : print('linear3: ', out.shape)
        #out = self.sigmoid(out)
        #if DEBUG_TRAIN : print('sigmoid: ', out.shape)
        if DEBUG_TRAIN : print()
        return out

> ## Create Model
> if saved model exist, load model from disk

In [10]:
model=CNN().double()
if(os.path.isfile(MODEL_PATH)):
    model.load_state_dict(torch.load(MODEL_PATH))
    print(MODEL_PATH, ' loaded!')
    model.eval()
    pass    
loss_function=nn.BCEWithLogitsLoss()
optimizer= torch.optim.Adam(model.parameters(),lr=learning_rate)

> ## Train Model

In [11]:
loss_arr=[]
for i in range(num_epoch):
    for j, sample in enumerate(dataloader):
        x = sample['image']
        y = sample['label']
        optimizer.zero_grad()
        output=model.forward(x)
        loss=loss_function(output,y)
        loss.backward()
        optimizer.step()
        if j == 0:
            if DEBUG_DATA:
                plt.figure(figsize=(32, 32))
                for k, img in enumerate(x):
                    plt.subplot(8, 8, k+1)
                    plt.imshow(img.squeeze())
                    plt.xticks([])
                    plt.yticks([])
                    pass
                plt.show()
            print('epoch: ', i+1, 'sequence: ', j, loss)
            loss_arr.append(loss.detach().numpy())

epoch:  1 sequence:  0 tensor(0.6945, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  2 sequence:  0 tensor(0.2905, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  3 sequence:  0 tensor(0.2285, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  4 sequence:  0 tensor(0.1724, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  5 sequence:  0 tensor(0.1491, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  6 sequence:  0 tensor(0.1359, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  7 sequence:  0 tensor(0.1306, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  8 sequence:  0 tensor(0.0998, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  9 sequence:  0 tensor(0.1124, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  10 sequence:  0 tensor(0.0934, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  11 sequence:  0 tensor(0.0827, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  12 sequence:  0 tensor(0.0822, grad_fn=<Bina

epoch:  96 sequence:  0 tensor(0.0217, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  97 sequence:  0 tensor(0.0141, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  98 sequence:  0 tensor(0.0157, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  99 sequence:  0 tensor(0.0198, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  100 sequence:  0 tensor(0.0083, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)


> ## Save Current Model

In [13]:
torch.save(model.state_dict(), MODEL_PATH)

> ## Test Model

In [14]:
with torch.no_grad():
    scoreboard = []
    for j, sample in enumerate(dataloader):
        x = sample['image']
        y = sample['label']
        output = model.forward(x)
        for i in range(len(output)):
            try:
                out = decode_from_bin2(output[i].tolist())
            except UnicodeDecodeError:
                out = 'UnicodeDecodeError'
            y_ = decode_from_bin(y[i].tolist())
            print(", 실제 값: ", y_, "출력 값: ", out)
            if out==y_:
                scoreboard.append(1)
            else:
                scoreboard.append(0)

, 실제 값:  겸 출력 값:  겸
, 실제 값:  챨 출력 값:  챨
, 실제 값:  횬 출력 값:  횬
, 실제 값:  일 출력 값:  일
, 실제 값:  쫘 출력 값:  쫘
, 실제 값:  똴 출력 값:  똴
, 실제 값:  튼 출력 값:  튼
, 실제 값:  흣 출력 값:  흣
, 실제 값:  갤 출력 값:  갤
, 실제 값:  눌 출력 값:  눌
, 실제 값:  븀 출력 값:  븀
, 실제 값:  텀 출력 값:  텀
, 실제 값:  색 출력 값:  샙
, 실제 값:  쩝 출력 값:  쩝
, 실제 값:  쐽 출력 값:  쐽
, 실제 값:  푭 출력 값:  푭
, 실제 값:  떱 출력 값:  떱
, 실제 값:  밞 출력 값:  밞
, 실제 값:  웹 출력 값:  웹
, 실제 값:  렸 출력 값:  렸
, 실제 값:  푿 출력 값:  푿
, 실제 값:  엥 출력 값:  엥
, 실제 값:  뫼 출력 값:  뫼
, 실제 값:  헉 출력 값:  헉
, 실제 값:  챈 출력 값:  챈
, 실제 값:  놂 출력 값:  놂
, 실제 값:  땋 출력 값:  땋
, 실제 값:  픈 출력 값:  픈
, 실제 값:  마 출력 값:  마
, 실제 값:  좟 출력 값:  좟
, 실제 값:  톨 출력 값:  톨
, 실제 값:  아 출력 값:  아
, 실제 값:  교 출력 값:  교
, 실제 값:  맨 출력 값:  맨
, 실제 값:  톺 출력 값:  톺
, 실제 값:  솎 출력 값:  솎
, 실제 값:  냄 출력 값:  냄
, 실제 값:  킷 출력 값:  킷
, 실제 값:  찬 출력 값:  찬
, 실제 값:  닻 출력 값:  닻
, 실제 값:  뎃 출력 값:  뎃
, 실제 값:  죠 출력 값:  죠
, 실제 값:  챕 출력 값:  챕
, 실제 값:  탉 출력 값:  탉
, 실제 값:  띕 출력 값:  띕
, 실제 값:  위 출력 값:  위
, 실제 값:  촨 출력 값:  촬
, 실제 값:  콴 출력 값:  콴
, 실제 값:  늅 출력 값:  늅
, 실제 값:  뺏 출력 값:  뺏


, 실제 값:  휵 출력 값:  휵
, 실제 값:  푠 출력 값:  푠
, 실제 값:  럿 출력 값:  럿
, 실제 값:  쉬 출력 값:  쉨
, 실제 값:  뭍 출력 값:  뭍
, 실제 값:  조 출력 값:  조
, 실제 값:  푀 출력 값:  푀
, 실제 값:  쵸 출력 값:  쵸
, 실제 값:  뻠 출력 값:  뻠
, 실제 값:  훑 출력 값:  훑
, 실제 값:  찹 출력 값:  찹
, 실제 값:  빎 출력 값:  빎
, 실제 값:  뎅 출력 값:  뎅
, 실제 값:  횰 출력 값:  횰
, 실제 값:  힁 출력 값:  힁
, 실제 값:  첸 출력 값:  첸
, 실제 값:  먁 출력 값:  먁
, 실제 값:  샌 출력 값:  샌
, 실제 값:  잎 출력 값:  잎
, 실제 값:  븜 출력 값:  븜
, 실제 값:  쏩 출력 값:  쏩
, 실제 값:  웸 출력 값:  웸
, 실제 값:  즙 출력 값:  즙
, 실제 값:  옰 출력 값:  옰
, 실제 값:  샘 출력 값:  샘
, 실제 값:  벴 출력 값:  벴
, 실제 값:  썲 출력 값:  썲
, 실제 값:  솩 출력 값:  솩
, 실제 값:  부 출력 값:  부
, 실제 값:  갓 출력 값:  갓
, 실제 값:  츠 출력 값:  츠
, 실제 값:  굄 출력 값:  굄
, 실제 값:  셰 출력 값:  셰
, 실제 값:  뤄 출력 값:  뤄
, 실제 값:  뜻 출력 값:  뜻
, 실제 값:  괘 출력 값:  괘
, 실제 값:  춈 출력 값:  초
, 실제 값:  삼 출력 값:  삼
, 실제 값:  댓 출력 값:  댓
, 실제 값:  풋 출력 값:  풋
, 실제 값:  땃 출력 값:  땃
, 실제 값:  증 출력 값:  증
, 실제 값:  뱉 출력 값:  뱉
, 실제 값:  평 출력 값:  평
, 실제 값:  텅 출력 값:  텅
, 실제 값:  군 출력 값:  군
, 실제 값:  먁 출력 값:  먁
, 실제 값:  씬 출력 값:  씬
, 실제 값:  떠 출력 값:  떠
, 실제 값:  흩 출력 값:  흩


, 실제 값:  땄 출력 값:  땄
, 실제 값:  잭 출력 값:  잡
, 실제 값:  춘 출력 값:  춘
, 실제 값:  체 출력 값:  체
, 실제 값:  줴 출력 값:  줴
, 실제 값:  튕 출력 값:  튕
, 실제 값:  꾹 출력 값:  꾹
, 실제 값:  퀄 출력 값:  퀄
, 실제 값:  췌 출력 값:  췌
, 실제 값:  앤 출력 값:  앤
, 실제 값:  꿱 출력 값:  꿱
, 실제 값:  휜 출력 값:  휜
, 실제 값:  틴 출력 값:  틴
, 실제 값:  틘 출력 값:  틘
, 실제 값:  항 출력 값:  항
, 실제 값:  헥 출력 값:  헥
, 실제 값:  읾 출력 값:  읾
, 실제 값:  텍 출력 값:  텍
, 실제 값:  네 출력 값:  네
, 실제 값:  셩 출력 값:  셩
, 실제 값:  리 출력 값:  리
, 실제 값:  듸 출력 값:  듸
, 실제 값:  읨 출력 값:  읨
, 실제 값:  샨 출력 값:  샨
, 실제 값:  꾹 출력 값:  꾹
, 실제 값:  뭘 출력 값:  뭘
, 실제 값:  매 출력 값:  매
, 실제 값:  짜 출력 값:  짜
, 실제 값:  궁 출력 값:  궁
, 실제 값:  첼 출력 값:  첼
, 실제 값:  튕 출력 값:  튕
, 실제 값:  뼈 출력 값:  뼈
, 실제 값:  텟 출력 값:  텟
, 실제 값:  련 출력 값:  련
, 실제 값:  닷 출력 값:  닷
, 실제 값:  띠 출력 값:  띠
, 실제 값:  밌 출력 값:  밌
, 실제 값:  눴 출력 값:  눴
, 실제 값:  걍 출력 값:  걍
, 실제 값:  짰 출력 값:  짰
, 실제 값:  끓 출력 값:  끓
, 실제 값:  채 출력 값:  채
, 실제 값:  명 출력 값:  명
, 실제 값:  떤 출력 값:  떤
, 실제 값:  훠 출력 값:  ퟠ
, 실제 값:  톤 출력 값:  톤
, 실제 값:  뭄 출력 값:  뭄
, 실제 값:  컫 출력 값:  컫
, 실제 값:  잡 출력 값:  잡
, 실제 값:  걍 출력 값:  걍


, 실제 값:  끄 출력 값:  끄
, 실제 값:  띵 출력 값:  띵
, 실제 값:  랍 출력 값:  랍
, 실제 값:  쇰 출력 값:  쇰
, 실제 값:  짢 출력 값:  짢
, 실제 값:  굳 출력 값:  굳
, 실제 값:  휘 출력 값:  휘
, 실제 값:  추 출력 값:  추
, 실제 값:  클 출력 값:  클
, 실제 값:  즌 출력 값:  즌
, 실제 값:  몫 출력 값:  몫
, 실제 값:  해 출력 값:  해
, 실제 값:  썲 출력 값:  썲
, 실제 값:  흑 출력 값:  흱
, 실제 값:  똴 출력 값:  똴
, 실제 값:  촹 출력 값:  촹
, 실제 값:  담 출력 값:  담
, 실제 값:  빕 출력 값:  빕
, 실제 값:  뀄 출력 값:  뀄
, 실제 값:  닥 출력 값:  닥
, 실제 값:  맞 출력 값:  맞
, 실제 값:  묻 출력 값:  묻
, 실제 값:  퀑 출력 값:  퀑
, 실제 값:  쩽 출력 값:  쩽
, 실제 값:  겜 출력 값:  겜
, 실제 값:  쉘 출력 값:  쉐
, 실제 값:  흼 출력 값:  흼
, 실제 값:  팔 출력 값:  팔
, 실제 값:  채 출력 값:  채
, 실제 값:  겠 출력 값:  겠
, 실제 값:  겸 출력 값:  겸
, 실제 값:  퀵 출력 값:  퀵
, 실제 값:  댁 출력 값:  댁
, 실제 값:  숑 출력 값:  숑
, 실제 값:  안 출력 값:  안
, 실제 값:  렐 출력 값:  렐
, 실제 값:  앗 출력 값:  앗
, 실제 값:  음 출력 값:  음
, 실제 값:  왁 출력 값:  왁
, 실제 값:  썬 출력 값:  썬
, 실제 값:  첨 출력 값:  첨
, 실제 값:  샵 출력 값:  샵
, 실제 값:  쫬 출력 값:  쫬
, 실제 값:  떡 출력 값:  떡
, 실제 값:  젱 출력 값:  젱
, 실제 값:  히 출력 값:  히
, 실제 값:  균 출력 값:  균
, 실제 값:  쌥 출력 값:  쌥
, 실제 값:  률 출력 값:  률
, 실제 값:  통 출력 값:  통


, 실제 값:  보 출력 값:  보
, 실제 값:  얗 출력 값:  얗
, 실제 값:  쑤 출력 값:  쑤
, 실제 값:  숌 출력 값:  숌
, 실제 값:  뮴 출력 값:  뮴
, 실제 값:  뇰 출력 값:  뇰
, 실제 값:  펐 출력 값:  펐
, 실제 값:  야 출력 값:  야
, 실제 값:  샜 출력 값:  샜
, 실제 값:  틂 출력 값:  틂
, 실제 값:  딱 출력 값:  딱
, 실제 값:  텁 출력 값:  텁
, 실제 값:  탓 출력 값:  탓
, 실제 값:  슁 출력 값:  슁
, 실제 값:  샐 출력 값:  샘
, 실제 값:  쿠 출력 값:  쿠
, 실제 값:  꽝 출력 값:  꽝
, 실제 값:  엮 출력 값:  엮
, 실제 값:  묠 출력 값:  묠
, 실제 값:  꽂 출력 값:  꽂
, 실제 값:  뮨 출력 값:  뮨
, 실제 값:  혜 출력 값:  혜
, 실제 값:  뵀 출력 값:  뵀
, 실제 값:  엌 출력 값:  엌
, 실제 값:  중 출력 값:  중
, 실제 값:  옙 출력 값:  옱
, 실제 값:  경 출력 값:  경
, 실제 값:  툭 출력 값:  툭
, 실제 값:  붙 출력 값:  붙
, 실제 값:  컫 출력 값:  컫
, 실제 값:  뜬 출력 값:  뜬
, 실제 값:  깨 출력 값:  깨
, 실제 값:  욍 출력 값:  욍
, 실제 값:  츙 출력 값:  츙
, 실제 값:  홉 출력 값:  홉
, 실제 값:  싸 출력 값:  싸
, 실제 값:  똑 출력 값:  똑
, 실제 값:  희 출력 값:  희
, 실제 값:  윤 출력 값:  윤
, 실제 값:  텝 출력 값:  텝
, 실제 값:  믈 출력 값:  믈
, 실제 값:  굶 출력 값:  굶
, 실제 값:  를 출력 값:  를
, 실제 값:  처 출력 값:  처
, 실제 값:  뽕 출력 값:  뽕
, 실제 값:  잽 출력 값:  잹
, 실제 값:  뇩 출력 값:  뇩
, 실제 값:  뀨 출력 값:  뀨
, 실제 값:  큠 출력 값:  큠
, 실제 값:  묏 출력 값:  묏


...

, 실제 값:  긺 출력 값:  긺
, 실제 값:  딜 출력 값:  딼
, 실제 값:  일 출력 값:  일
, 실제 값:  감 출력 값:  감
, 실제 값:  샥 출력 값:  샥
, 실제 값:  앨 출력 값:  앨
, 실제 값:  싣 출력 값:  싣
, 실제 값:  팹 출력 값:  팹
, 실제 값:  쿠 출력 값:  쿠
, 실제 값:  대 출력 값:  대
, 실제 값:  뚱 출력 값:  뚱
, 실제 값:  뜸 출력 값:  뜸
, 실제 값:  랴 출력 값:  랴
, 실제 값:  돨 출력 값:  돨
, 실제 값:  뉴 출력 값:  뉴
, 실제 값:  뒵 출력 값:  뒵
, 실제 값:  틥 출력 값:  틥
, 실제 값:  첫 출력 값:  첫
, 실제 값:  깩 출력 값:  깩
, 실제 값:  밭 출력 값:  밭
, 실제 값:  홈 출력 값:  홈
, 실제 값:  쥣 출력 값:  쥣
, 실제 값:  쭸 출력 값:  쭸
, 실제 값:  레 출력 값:  레
, 실제 값:  뜩 출력 값:  뜉
, 실제 값:  괬 출력 값:  괬
, 실제 값:  쌥 출력 값:  쌥
, 실제 값:  왝 출력 값:  왝
, 실제 값:  빤 출력 값:  비
, 실제 값:  녠 출력 값:  녠
, 실제 값:  노 출력 값:  노
, 실제 값:  솰 출력 값:  솰
, 실제 값:  을 출력 값:  음
, 실제 값:  맥 출력 값:  맥
, 실제 값:  왐 출력 값:  왐
, 실제 값:  묵 출력 값:  묵
, 실제 값:  교 출력 값:  교
, 실제 값:  열 출력 값:  열
, 실제 값:  댕 출력 값:  댕
, 실제 값:  못 출력 값:  못
, 실제 값:  뒝 출력 값:  뒝
, 실제 값:  탕 출력 값:  탕
, 실제 값:  꿜 출력 값:  꿜
, 실제 값:  녈 출력 값:  녈
, 실제 값:  익 출력 값:  익
, 실제 값:  뻔 출력 값:  뻔
, 실제 값:  뢨 출력 값:  뢨
, 실제 값:  죠 출력 값:  죠
, 실제 값:  른 출력 값:  른
, 실제 값:  낡 출력 값:  낡


, 실제 값:  센 출력 값:  센
, 실제 값:  짖 출력 값:  짒
, 실제 값:  괩 출력 값:  괩
, 실제 값:  촹 출력 값:  촹
, 실제 값:  엘 출력 값:  엘
, 실제 값:  삠 출력 값:  삠
, 실제 값:  땐 출력 값:  땐
, 실제 값:  갓 출력 값:  갓
, 실제 값:  틜 출력 값:  틜
, 실제 값:  욥 출력 값:  욵
, 실제 값:  겁 출력 값:  겁
, 실제 값:  쥐 출력 값:  쥐
, 실제 값:  쟤 출력 값:  쟤
, 실제 값:  쵸 출력 값:  쵸
, 실제 값:  선 출력 값:  선
, 실제 값:  퀸 출력 값:  퀸
, 실제 값:  찍 출력 값:  찍
, 실제 값:  살 출력 값:  살
, 실제 값:  갉 출력 값:  갉
, 실제 값:  되 출력 값:  되
, 실제 값:  땁 출력 값:  땁
, 실제 값:  니 출력 값:  냈
, 실제 값:  쥘 출력 값:  쥘
, 실제 값:  뗄 출력 값:  뗄
, 실제 값:  쮜 출력 값:  쮜
, 실제 값:  틋 출력 값:  틋
, 실제 값:  거 출력 값:  거
, 실제 값:  멜 출력 값:  메
, 실제 값:  짙 출력 값:  짙
, 실제 값:  덜 출력 값:  덜
, 실제 값:  빻 출력 값:  빻
, 실제 값:  컹 출력 값:  컹
, 실제 값:  찔 출력 값:  찜
, 실제 값:  켈 출력 값:  켈
, 실제 값:  뒈 출력 값:  뒈
, 실제 값:  밀 출력 값:  밀
, 실제 값:  홍 출력 값:  흍
, 실제 값:  닦 출력 값:  닦
, 실제 값:  끙 출력 값:  끙
, 실제 값:  긺 출력 값:  긺
, 실제 값:  빴 출력 값:  빔
, 실제 값:  쥘 출력 값:  쥘
, 실제 값:  빅 출력 값:  빅
, 실제 값:  튀 출력 값:  튄
, 실제 값:  팔 출력 값:  팔
, 실제 값:  보 출력 값:  보
, 실제 값:  닙 출력 값:  닙
, 실제 값:  뺨 출력 값:  뺨
, 실제 값:  낱 출력 값:  낱
, 실제 값:  챈 출력 값:  챈


, 실제 값:  롑 출력 값:  롑
, 실제 값:  랍 출력 값:  랍
, 실제 값:  쳉 출력 값:  쳉
, 실제 값:  늬 출력 값:  늬
, 실제 값:  카 출력 값:  카
, 실제 값:  딸 출력 값:  딸
, 실제 값:  휀 출력 값:  휐
, 실제 값:  구 출력 값:  구
, 실제 값:  카 출력 값:  카
, 실제 값:  뎔 출력 값:  뎔
, 실제 값:  춈 출력 값:  춈
, 실제 값:  웹 출력 값:  웹
, 실제 값:  췹 출력 값:  췹
, 실제 값:  꾄 출력 값:  꾄
, 실제 값:  예 출력 값:  예
, 실제 값:  쓿 출력 값:  쓿
, 실제 값:  차 출력 값:  찬
, 실제 값:  쫌 출력 값:  쫌
, 실제 값:  부 출력 값:  부
, 실제 값:  솅 출력 값:  솁
, 실제 값:  펀 출력 값:  펀
, 실제 값:  몃 출력 값:  몃
, 실제 값:  덩 출력 값:  덩
, 실제 값:  튜 출력 값:  튌
, 실제 값:  뭅 출력 값:  뭅
, 실제 값:  뭄 출력 값:  뭄
, 실제 값:  피 출력 값:  피
, 실제 값:  빤 출력 값:  빤
, 실제 값:  싶 출력 값:  싶
, 실제 값:  틂 출력 값:  틂
, 실제 값:  겟 출력 값:  겟
, 실제 값:  꽥 출력 값:  꽥
, 실제 값:  좋 출력 값:  좋
, 실제 값:  좼 출력 값:  좼
, 실제 값:  등 출력 값:  등
, 실제 값:  쎌 출력 값:  쎌
, 실제 값:  볶 출력 값:  볶
, 실제 값:  뿜 출력 값:  뿜
, 실제 값:  똬 출력 값:  똬
, 실제 값:  갱 출력 값:  갱
, 실제 값:  틜 출력 값:  틜
, 실제 값:  튑 출력 값:  튑
, 실제 값:  솖 출력 값:  솖
, 실제 값:  샷 출력 값:  샷
, 실제 값:  뒷 출력 값:  뒷
, 실제 값:  쉑 출력 값:  쉑
, 실제 값:  왹 출력 값:  왹
, 실제 값:  앉 출력 값:  앉
, 실제 값:  펴 출력 값:  펰
, 실제 값:  칠 출력 값:  칠


, 실제 값:  쁠 출력 값:  쁨
, 실제 값:  샴 출력 값:  샴
, 실제 값:  은 출력 값:  은
, 실제 값:  찾 출력 값:  찾
, 실제 값:  윽 출력 값:  윽
, 실제 값:  숱 출력 값:  숱
, 실제 값:  떱 출력 값:  떱
, 실제 값:  흖 출력 값:  흖
, 실제 값:  얠 출력 값:  얠
, 실제 값:  뽑 출력 값:  뽁
, 실제 값:  띤 출력 값:  띤
, 실제 값:  푤 출력 값:  푤
, 실제 값:  빡 출력 값:  빡
, 실제 값:  큭 출력 값:  큽
, 실제 값:  췄 출력 값:  췤
, 실제 값:  됴 출력 값:  됴
, 실제 값:  낯 출력 값:  낯
, 실제 값:  황 출력 값:  황
, 실제 값:  빕 출력 값:  빕
, 실제 값:  칵 출력 값:  칵
, 실제 값:  냘 출력 값:  냘
, 실제 값:  서 출력 값:  서
, 실제 값:  류 출력 값:  류
, 실제 값:  뉵 출력 값:  뉵
, 실제 값:  쿳 출력 값:  쿳
, 실제 값:  끅 출력 값:  끅
, 실제 값:  괼 출력 값:  괼
, 실제 값:  숌 출력 값:  숌
, 실제 값:  송 출력 값:  송
, 실제 값:  읓 출력 값:  읓
, 실제 값:  츠 출력 값:  츠
, 실제 값:  땃 출력 값:  땃
, 실제 값:  킁 출력 값:  킁
, 실제 값:  울 출력 값:  울
, 실제 값:  실 출력 값:  실
, 실제 값:  긷 출력 값:  긷
, 실제 값:  곡 출력 값:  곡
, 실제 값:  후 출력 값:  후
, 실제 값:  꿎 출력 값:  꿊
, 실제 값:  잔 출력 값:  임
, 실제 값:  섐 출력 값:  섐
, 실제 값:  쉰 출력 값:  쉰
, 실제 값:  댁 출력 값:  댁
, 실제 값:  뢨 출력 값:  뢨
, 실제 값:  쐽 출력 값:  쐽
, 실제 값:  쟘 출력 값:  쟘
, 실제 값:  깐 출력 값:  깐
, 실제 값:  쥣 출력 값:  쥳
, 실제 값:  섬 출력 값:  섬
, 실제 값:  풂 출력 값:  풂


, 실제 값:  믈 출력 값:  믈
, 실제 값:  탁 출력 값:  탁
, 실제 값:  웸 출력 값:  웸
, 실제 값:  얹 출력 값:  얹
, 실제 값:  벰 출력 값:  벰
, 실제 값:  쬘 출력 값:  쬘
, 실제 값:  왹 출력 값:  웹
, 실제 값:  샥 출력 값:  샥
, 실제 값:  뮐 출력 값:  뮐
, 실제 값:  밝 출력 값:  밝
, 실제 값:  샹 출력 값:  샹
, 실제 값:  띰 출력 값:  띰
, 실제 값:  뽐 출력 값:  뽐
, 실제 값:  꿀 출력 값:  꿀
, 실제 값:  훵 출력 값:  훵
, 실제 값:  삶 출력 값:  삶
, 실제 값:  션 출력 값:  션
, 실제 값:  쑴 출력 값:  쑴
, 실제 값:  촨 출력 값:  촨
, 실제 값:  훽 출력 값:  훽
, 실제 값:  굔 출력 값:  굔
, 실제 값:  츰 출력 값:  츰
, 실제 값:  후 출력 값:  후
, 실제 값:  쌨 출력 값:  쌨
, 실제 값:  뺏 출력 값:  뺏
, 실제 값:  약 출력 값:  앵
, 실제 값:  왁 출력 값:  왁
, 실제 값:  깁 출력 값:  깁
, 실제 값:  네 출력 값:  네
, 실제 값:  섯 출력 값:  섯
, 실제 값:  몸 출력 값:  몸
, 실제 값:  럇 출력 값:  럇
, 실제 값:  읾 출력 값:  읾
, 실제 값:  줴 출력 값:  줴
, 실제 값:  싶 출력 값:  싶
, 실제 값:  갱 출력 값:  갱
, 실제 값:  콴 출력 값:  콴
, 실제 값:  잘 출력 값:  잘
, 실제 값:  뇝 출력 값:  뇕
, 실제 값:  밭 출력 값:  밭
, 실제 값:  빰 출력 값:  빰
, 실제 값:  쩝 출력 값:  쩝
, 실제 값:  매 출력 값:  매
, 실제 값:  뵤 출력 값:  뵤
, 실제 값:  칸 출력 값:  칸
, 실제 값:  륵 출력 값:  륵
, 실제 값:  죡 출력 값:  죡
, 실제 값:  횻 출력 값:  횻
, 실제 값:  솨 출력 값:  솨
, 실제 값:  쫬 출력 값:  쫬


In [15]:
sum=0
for i in range(len(scoreboard)):
    sum+=int(scoreboard[i])
performance=sum/len(scoreboard)*100
print("performance: ",performance,"%")

performance:  91.26757566623522 %
